In [1]:
import pandas as pd
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
train = train[~pd.isnull(train['document'])]
train_text = train['document'].values.tolist()

batch_size = 50
all_texts = [train_text[i : i + batch_size] for i in range(0, len(train_text), batch_size)]

def batch_iterator():
    for i in range(0, len(train_text), batch_size):
        yield train_text[i : i + batch_size]

In [2]:
from transformers import AutoTokenizer
MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [3]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [4]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=40000)

In [5]:
import os

os.mkdir('./tokenizer_practice')
new_tokenizer.save_pretrained('./tokenizer_practice')

## 알 수 있듯이 distilbert의 경우는 vocab.txt를 저장한다. 

('./tokenizer_practice\\tokenizer_config.json',
 './tokenizer_practice\\special_tokens_map.json',
 './tokenizer_practice\\vocab.txt',
 './tokenizer_practice\\added_tokens.json',
 './tokenizer_practice\\tokenizer.json')

In [6]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()
tokenizer.train('nsmc/ratings_train.txt', vocab_size=150, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [7]:
tokenizer ## 보면 알 수 있듯이 vocab_size를 150으로 했지만 알고리즘에 의해서 최소 261개가 된다.

Tokenizer(vocabulary_size=261, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [9]:
tokenizer.save_model('./tokenizer_practice')  ## Bytelevel의 경우에는 vocab.json과 merges.txt가 생긴다. 위에서 vocab.txt와는 상반된다. 
## 해당되는 pretrained tokenizer와 접합시키기 위해서는 pretrained tokenize가 vocab.json인지 vocab.txt를 쓰는지 확인해야한다. 

['./tokenizer_practice\\vocab.json', './tokenizer_practice\\merges.txt']

---
### 자신만의 tokenizer 만들기

In [10]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))
tokenizer.get_vocab_size()

0

In [11]:
### normalize pre_tokenize를 각각지정해준다. pretokenize는 tokenize 전단계를 의미한다. 헷갈리지 말자. 
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
tokenizer.normalizer.normalize_str('I HAVE A dOG')

'i have a dog'

In [12]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [13]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example OF THE WORLD")

[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('OF', (19, 21)),
 ('THE', (22, 25)),
 ('WORLD', (26, 31))]

In [14]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=1200, special_tokens=special_tokens)

### 이건 special token이라는걸 지정만 해줄뿐 각각이 뭔지를 나타내는건 아니다. vocab에다가 추가하려고 하는것!!
### vocab_size도 최소라는게 있는것같다. 800보다 작게 하면 무조건 800으로 뜨고 1200으로 하면 1200으로 뜬다. 

In [16]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [17]:
tokenizer.get_vocab_size()

1200

In [18]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [21]:
tokenizer.cls_token_id## 아직까지는 cls 토큰이라는게 없다. 

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'cls_token_id'

In [22]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [23]:
encoder = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

In [24]:
encoder.tokens

['[CLS]',
 't',
 '##h',
 '##i',
 '##s',
 'i',
 '##s',
 'o',
 '##n',
 '##e',
 's',
 '##e',
 '##n',
 '##t',
 '##e',
 '##n',
 '##c',
 '##e',
 '.',
 '[SEP]',
 'w',
 '##i',
 '##t',
 '##h',
 't',
 '##h',
 '##i',
 '##s',
 'o',
 '##n',
 '##e',
 'w',
 '##e',
 'h',
 '##a',
 '##v',
 '##e',
 'a',
 'p',
 '##a',
 '##i',
 '##r',
 '.',
 '[SEP]']

In [ ]:
### 구글에서는 BERT 학습할때 WPM 공개안했지만, HUGGINGFACE에서 공개했다고 함. 이건 HUGGINGFACE 사용하는 방법. 다른사람거. 
### huggingface로 wordpiece기반의 custom tokenizer만드는것. 

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
import os


def parse_corpus_path():
    corpus_paths = []
    for current, dirs, files in os.walk("corpus"):
        for file in files:
            corpus_paths.append(os.path.join(current, file))
    return corpus_paths


def WPM(size, corpora):
    wpm_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
    trainer = WordPieceTrainer(
        vocab_size=size, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
    )
    wpm_tokenizer.train(corpora, trainer)
    wpm_tokenizer.save("vocab/WPM_everyone.json")


def main():
    corpus_paths = ["./corpus_pp.txt"]
    WPM(32000, corpus_paths)


if __name__ == '__main__':
    main()